# VGG16_LargeFOV

## 1. Environment Setup

### 1.0. Check GPU


In [1]:
from data import LungImageDataset, colorDict
from torch.utils.data import DataLoader
# import augmentation
import model
import train

import torch
import torchvision

# Ignore Warning
import warnings

warnings.filterwarnings(action="ignore")

test


2024-01-17 17:32:44.790521: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-17 17:32:44.793535: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 17:32:44.855770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 17:32:44.855819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 17:32:44.857161: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
use_colab = False
use_gpu = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
if use_colab:
    ! git clone https://github.com/chendonghp/deeplab-v1.git
    !pip install -U cython
    !pip install git+https://github.com/lucasb-eyer/pydensecrf.git

    from google.colab import drive
    drive.mount('/content/drive')

    %cd deeplab-v1/Implementation/
    ! unzip /content/drive/MyDrive/data.zip -d ./dataset
    use_gpu = True

In [4]:
! ls

'DeepLab_v1 - SBD, VOC 2012.ipynb'    __pycache__       experiment   utils.py
 Inference.ipynb		      augmentation.py   model.py
 VGG16_LargeFOV_SBD,_VOC_2012.ipynb   best.pt	        test.py
 __init__.py			      data.py	        train.py


In [5]:
if use_colab:
    data_root = r"./dataset"

    # dataset
    train_batch_size = 130
    test_batch_size = 60
    train_ratio = 0.9
    size = 12000  # select num of images to put in dataset
    train_size = int(size * train_ratio)
    train_range, val_range = (0, train_size), (train_size, size)

    # model parameters
    num_classes = len(colorDict.keys())
    ignore_index = 255

    # train hyperparameters
    epochs = 100
    lr = 0.001
    momentum = 0.9
    weight_decay = 0.0005
    init_weights = True

    # log and save
    print_freq = 5
    epoch_print = 1
    path = "/content/drive/MyDrive/experiment/vgg16_largefov"
    load_path = path
    save_path = path
    log_path = path
else:
    data_root = r"/mnt/d/data/"
    # dataset
    train_batch_size = 10
    test_batch_size = 10
    train_ratio = 0.9
    size = 120  # select num of images to put in dataset
    train_size = int(size * train_ratio)
    train_range, val_range = (0, train_size), (train_size, size)

    # model parameters
    num_classes = len(colorDict.keys())
    ignore_index = 255

    # train hyperparameters
    epochs = 20
    lr = 0.001
    momentum = 0.9
    weight_decay = 0.0005
    init_weights = True

    # log and save
    print_freq = 5
    epoch_print = 1
    path = "experiment/vgg16_largefov"
    load_path = path
    save_path = path
    log_path = path

import os

if not os.path.exists(path):
    os.makedirs(path)

## 2. Data Preprocessing


In [6]:
# compute image mean and std
from data import compute_mean_std
import json
# Using Albumentations library for data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2


dataset = LungImageDataset(
    data_root,
    transform=A.Compose(
        [ToTensorV2()]
    ),
    size=(0, size),
)
dataset_loader = DataLoader(
    dataset,
    batch_size=200,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

if not os.path.exists("config.json"):
    print("config.json file does not exist, compute now")
    mean, std = compute_mean_std(dataset_loader)
    mean, std = mean.tolist(), std.tolist()
    data = {"mean": mean, "std": std}
    with open("config.json", "w") as f:
        json.dump(data, f)
    print("Mean:", mean)
    print("Std:", std)
else:
    with open("config.json", "r") as f:
        data = json.load(f)

    mean, std = data["mean"], data["std"]
    print("Mean:", mean)
    print("Std:", std)

config.json file does not exist, compute now
Mean: [191.48016357421875, 141.48892211914062, 183.9773406982422]
Std: [50.47390365600586, 68.20906829833984, 44.792720794677734]


In [11]:

train_tf = A.Compose(
    [A.Resize(256, 256), A.RandomCrop(256, 256), A.HorizontalFlip(), A.Normalize(mean=mean, std=mean), ToTensorV2(True)]
)

val_tf = A.Compose([A.Resize(256, 256), A.Normalize(mean=mean, std=mean), ToTensorV2(True)])

In [12]:
train_dataset = LungImageDataset(data_root, size=train_range, transform=train_tf)
val_dataset = LungImageDataset(data_root, size=val_range, transform=val_tf)


train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=test_batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

# train_dataset = torchvision.datasets.SBDataset(root='./', image_set='train_noval', mode='segmentation', download=False, transforms=train_tf)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)

# val_dataset = torchvision.datasets.VOCSegmentation(root='./', year='2012', image_set='val', download=False, transforms=val_tf)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [13]:
train_features, train_labels = next(iter(train_loader))
train_features.shape, train_labels.shape

(torch.Size([10, 3, 256, 256]), torch.Size([10, 1, 256, 256]))

## 3. Train Model


In [14]:
vgg16_largefov = train.VGG16_LargeFOV(
    num_classes=num_classes,
    init_weights=init_weights,
    ignore_index=ignore_index,
    use_gpu=use_gpu,
    device=device,
    print_freq=print_freq,
    epoch_print=epoch_print,
)

In [15]:
if use_colab:
    %load_ext tensorboard
    %tensorboard --logdir  {path}/runs
else:
    %load_ext tensorboard
    %tensorboard --logdir  {path}/runs

Launching TensorBoard...

In [16]:
vgg16_largefov.train(
    train_loader,
    val_loader,
    load_path=None,
    save_path=save_path,
    log_path=log_path,
    epochs=epochs,
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay,
)

Epoch 0 Started...
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.

 *********************************** Best mIoU Updated ***********************************
Iteration : 0 - Train Loss : 1.791929, Test Loss : 1.793070, Test mIoU : 3.0458, Test mpa : 3.4965
Saved Model at experiment/vgg16_largefov/vgg16_large_fov_best.pt.

Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.
Iteration : 4 - Train Loss : 1.788671, Test Loss : 1.791071, Test mIoU : 2.4045, Test mpa : 2.6342
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_log.csv.
Training log saved to experiment/vgg16_largefov/vgg_largefov_training_l